This example loads the prescription of the afocal telescope for the PICTURE-C balloon and runs it for a variety of array sizes to benchmark performance.

For more detailed explanations see Douglas et al. Proc. SPIE 2015 and Mendillo et al. Proc. SPIE 2015.

In [26]:
%matplotlib inline
#load modules
import numpy as np
import matplotlib.pyplot as plt

from astropy import units as u
from astropy.io import fits

import poppy
from poppy import fresnel
import logging
import os, platform, subprocess, re



In [38]:
%%bash
#get CPU model and total RAM:
#print system info, the reference system is a 13" 2012 Macbook Pro with:
#Intel(R) Core(TM) i5-3210M CPU @ 2.50GHz
#hw.memsize: 8589934592

sysctl -n machdep.cpu.brand_string  #for Macs
sysctl hw.memsize #for Macs

cat /proc/cpuinfo | grep model #for Linux
cat /proc/meminfo  | grep MemTotal #for Linux



Intel(R) Core(TM) i5-3210M CPU @ 2.50GHz
hw.memsize: 8589934592


cat: /proc/cpuinfo: No such file or directory
cat: /proc/meminfo: No such file or directory


In [13]:
npix=1024

#import data in inches:
inches=u.imperial.inch
sys=np.genfromtxt('data/rx_afocal_old.csv',delimiter=',', 
                  dtype="S10,S10,f8,f8,f8,f8,f8,f8,f8,f8,f8,f8,f8,f8,S5,S5,",
                  skip_header=16,names=True)

primary_r=sys['Radius_in'][0]*inches
primary_fl=sys['Focal_Length_in'][0]*inches

pic_c_sys = poppy.FresnelOpticalSystem(pupil_diameter=2*primary_r, npix=npix,beam_ratio=.34)

entrance=poppy.CircularAperture(radius=primary_r.to(u.m).value)
pic_c_sys.add_optic(entrance)
for n_optic,optic in enumerate(sys):

    fl=optic['Focal_Length_in'] *inches
    
    #roll the distances by one, and don't propagate after the last optic
    # Since the POPPY system convention is the location of the optic 
    #and the PICSIM convention is the distance after the optic,
    dz = sys[n_optic]['Distance_in'] * inches

    if (n_optic == 0): 
        dz_sys = 0*u.m
    else:
        dz_sys = sys[n_optic-1]['Distance_in'] * inches

    if fl !=0:
        gl=fresnel.GaussianLens(fl,name=optic['Name'])
        pic_c_sys.add_optic(gl,distance=dz_sys)
        pic_c_sys.add_optic(poppy.CircularAperture(radius=(optic['Radius_in']*inches).to(u.m).value))
    elif n_optic == sys.size-1:
        #if focal lengths worked, this could be an image
        pic_c_sys.add_optic(poppy.ScalarTransmission(planetype=poppy.PlaneType.intermediate,
                                                     name=optic['Name']), distance=dz_sys)
        #image plane doesn't get a circular aperture
    else:
        pic_c_sys.add_optic(poppy.ScalarTransmission(planetype=poppy.PlaneType.intermediate,
                                                     name=optic['Name']), distance=dz_sys)
        pic_c_sys.add_optic(poppy.CircularAperture(radius=(optic['Radius_in']*inches).to(u.m).value))
    #mask the beam by the size of the optic

    

In [14]:
poppy.conf.use_fftw=True
%timeit pic_c_sys.calcPSF(wavelength=675e-9,display_intermediates=False,return_intermediates=False)
#1 loops, best of 3: 1min 54s per loop


1 loops, best of 3: 1min 54s per loop


In [15]:
#generally, np.fft is faster for small arrays, since it has less overhead, and pyfftw is faster for big arrays
poppy.conf.use_fftw=False
%timeit pic_c_sys.calcPSF(wavelength=675e-9,display_intermediates=False,return_intermediates=False)


1 loops, best of 3: 1min 41s per loop


In [6]:
#should take the same amount of time if returning intermediate arrays
%time pic_c_sys.calcPSF(wavelength=675e-9,display_intermediates=False,return_intermediates=True)
psf,waves=pic_c_sys.calcPSF(wavelength=675e-9,display_intermediates=False,return_intermediates=True)


CPU times: user 3.91 s, sys: 746 ms, total: 4.65 s
Wall time: 4.64 s


In [9]:
pic_c_sys.npix=128
poppy.conf.use_fftw=True
%timeit pic_c_sys.calcPSF(wavelength=675e-9,display_intermediates=False,return_intermediates=False)
poppy.conf.use_fftw=False
%timeit pic_c_sys.calcPSF(wavelength=675e-9,display_intermediates=False,return_intermediates=False)

1 loops, best of 3: 1.08 s per loop
1 loops, best of 3: 1.06 s per loop
